In [ ]:
import os

from pathlib import Path

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib as mpl
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

from sdm_eurec4a.visulization import set_custom_rcParams, adjust_lightness_array

from sdm_eurec4a.identifications import match_clouds_and_cloudcomposite, match_clouds_and_dropsondes

from sdm_eurec4a import get_git_revision_hash, RepositoryPath

In [ ]:
plt.style.use("default")
default_colors = set_custom_rcParams()
darker_colors = adjust_lightness_array(default_colors, 0.75)

REPOSITORY_ROOT = RepositoryPath("levante").get_repo_dir()

output_dir = REPOSITORY_ROOT / Path("data/model/input_examples/")
# output_dir.mkdir(parents=True, exist_ok=True)

fig_path = REPOSITORY_ROOT / Path("results/compare_cluster_and_clouds/")
fig_path.mkdir(parents=True, exist_ok=True)

In [ ]:
clusters = xr.open_dataset(
    REPOSITORY_ROOT
    / Path(
        "data/observation/cloud_composite/processed/identified_clouds/identified_clusters_rain_mask_5.nc"
    )
)
clouds = xr.open_dataset(
    REPOSITORY_ROOT
    / Path("data/observation/cloud_composite/processed/identified_clouds/identified_clouds_rain_mask.nc")
)

# select only clouds which are between 800 and 1100 m
clouds = clouds.where(
    (clouds.alt >= 300) & (clouds.alt <= 1100) & (clouds.vertical_extent < 100), drop=True
)
clusters = clusters.where(
    (clusters.alt >= 300) & (clusters.alt <= 1100) & (clusters.vertical_extent < 100), drop=True
)

distance_clouds = xr.open_dataset(
    REPOSITORY_ROOT
    / Path(f"data/observation/combined/distance/distance_dropsondes_identified_clouds_rain_mask.nc")
)

distance_clusters = xr.open_dataset(
    REPOSITORY_ROOT
    / Path(f"data/observation/combined/distance/distance_dropsondes_identified_clusters_rain_mask_5.nc")
)

cloud_composite = xr.open_dataset(
    REPOSITORY_ROOT / Path("data/observation/cloud_composite/processed/cloud_composite_si_units.nc"),
    chunks={"time": 1000},
)


drop_sondes = xr.open_dataset(
    REPOSITORY_ROOT / Path("data/observation/dropsonde/processed/drop_sondes.nc")
)

In [ ]:
cloud_composite.cloud_mask

In [ ]:
fig, axs = plt.subplots(1, 2)
axs[0].scatter(
    clouds["duration"].dt.seconds,
    clouds["alt"],
    clouds["liquid_water_content"],
    marker="o",
    linestyle="None",
    color=default_colors[0],
    label="Clouds",
)
axs[0].scatter(
    clusters["duration"].dt.seconds,
    clusters["alt"],
    clusters["liquid_water_content"],
    marker="o",
    linestyle="None",
    color=default_colors[1],
    label="Clusters",
)
axs[0].set_xlabel("Duration in s")
axs[0].set_ylabel("Altitude in m")
# axs[0].set_title("LWC in kg/kg")


axs[1].scatter(
    clouds["horizontal_extent"],
    clouds["vertical_extent"],
    clouds["liquid_water_content"],
    marker="o",
    linestyle="None",
    color=default_colors[0],
    label="Clouds",
)
axs[1].scatter(
    clusters["horizontal_extent"],
    clusters["vertical_extent"],
    clusters["liquid_water_content"],
    marker="o",
    linestyle="None",
    color=default_colors[1],
    label="Clusters",
)
axs[1].set_xlabel("Horizontal extent in m")
axs[1].set_ylabel("Vertical extent in m")
# axs[1].set_title("LWC in kg/kg")

for ax in axs.flatten():
    ax.legend()

annotation_number = 5

for t in clouds.sortby("liquid_water_content").time[-annotation_number:]:
    subdata = clouds.sel(time=t)
    axs[0].annotate(
        subdata.cloud_id.data,
        (subdata.duration.dt.seconds, subdata.alt),
        color=darker_colors[0],
    )
    axs[1].annotate(
        subdata.cloud_id.data,
        (subdata["horizontal_extent"], subdata["vertical_extent"]),
        color=darker_colors[0],
    )
for t in clusters.sortby("liquid_water_content").time[-annotation_number:]:
    subdata = clusters.sel(time=t)
    axs[0].annotate(
        subdata.cloud_id.data,
        (subdata.duration.dt.seconds, subdata.alt),
        color=darker_colors[1],
    )

    axs[1].annotate(
        subdata.cloud_id.data,
        (subdata["horizontal_extent"], subdata["vertical_extent"]),
        color=darker_colors[1],
    )
# fig.savefig(fig_path / Path("cloud_properties.png"))

In [ ]:
import pandas as pd

In [ ]:
from sdm_eurec4a.identifications import select_individual_cloud_by_id

select_individual_cloud_by_id(clusters, chosen_id=213).time

In [ ]:
time_slice = slice(
    pd.to_datetime("2020-02-11 T 07:57"),
    # pd.to_datetime("2020-02-11 T 07:57:40")
    pd.to_datetime("2020-02-11 T 08:01"),
)

example_cloud = clouds.sel(time=time_slice)
example_cluster = clusters.sel(time=time_slice)

cloud_ATR = match_clouds_and_cloudcomposite(example_cloud, cloud_composite)

cluster_ATR = match_clouds_and_cloudcomposite(example_cluster, cloud_composite)


full_ATR = cloud_composite.sel(time=time_slice)

Create plot which compares cloud and cloud clusters

1. Cloud properties (Dots for height of ATR flight, Filled bars where the mask is true)
2. Cluster properties (as 1.)
3. Pcolormesh of PSD
4. LWC for rain and all droplets


In [ ]:
fig = plt.figure(figsize=(16, 9))
outer_grid = fig.add_gridspec(ncols=52, nrows=4, wspace=0, hspace=0.5)

ax1 = fig.add_subplot(outer_grid[0, 3:50])
ax2 = fig.add_subplot(outer_grid[1, 3:50], sharex=ax1, sharey=ax1)
ax3 = fig.add_subplot(outer_grid[2, 3:50], sharex=ax1)
cax = fig.add_subplot(outer_grid[2, 51])
ax4 = fig.add_subplot(outer_grid[3, 3:50], sharex=ax1)
# ax = fig.add_subplot(outer_grid[0, :])

ax1_title = fig.add_subplot(outer_grid[0, :], frame_on=False)
ax1_title.axis("off")
ax1_title.set_ylabel("Clouds", fontsize=16)

ax1.scatter(cloud_ATR.time, cloud_ATR.alt, marker=".", color=darker_colors[0], label="ATR measurments")
ax2.scatter(
    cluster_ATR.time, cluster_ATR.alt, marker=".", color=darker_colors[1], label="ATR measurments"
)
yvalues = [700, 760]

for t, s, e in zip(example_cloud.time, example_cloud.start, example_cloud.end):
    line = ax1.axvline(t.data, ymin=0, ymax=1, color=darker_colors[0], linewidth=1, linestyle="--")
    fill = ax1.fill_betweenx(yvalues, s.data, e.data, color=default_colors[0], alpha=0.5)
line.set_label("Mid time")
fill.set_label("Duration")
for t, s, e in zip(example_cluster.time, example_cluster.start, example_cluster.end):
    line = ax2.axvline(t.data, ymin=0, ymax=1, color=darker_colors[1], linewidth=1, linestyle="--")
    fill = ax2.fill_betweenx(yvalues, s.data, e.data, color=default_colors[1], alpha=0.5)
line.set_label("Mid time")
fill.set_label("Duration")

# ax1.set_title("Identified Clouds", y=1.0, pad=-14)
ax1.set_ylabel("Clouds\n\nAltitude in m")
ax1.legend()
# ax2.set_title("Identified Cloud Clusters", y=1.0, pad=-14)
ax2.set_ylabel("Cloud Clusters\n\nAltitude in m")
ax2.legend()

vmin, vmax = 1, 1e8
pm = ax3.pcolormesh(
    full_ATR.time,
    full_ATR.radius,
    full_ATR.particle_size_distribution,
    cmap="Greys",
    norm=mpl.colors.LogNorm(vmin, vmax),
)
split_radius = 4.5e-5
ax3.axhline(
    split_radius, color="r", linestyle="--", linewidth=1, label="Split between cloud and rain droplets"
)

ax3.set_yscale("log")
# ax3.set_xlabel("Time")
ax3.set_ylabel("PSD\n\nRadius in m")
# ax3.set_title("PSD over time", y=1.0, pad=-14)
ax3.legend()
fig.colorbar(pm, cax=cax, orientation="vertical", label=r"Drop counts in #$/m^{-3}$")

ax4.plot(
    full_ATR.time,
    1e3 * full_ATR["mass_size_distribution"].sel(radius=slice(split_radius, None)).sum(dim="radius"),
    color=default_colors[2],
    label="Rain drops only",
)
ax4.plot(
    full_ATR.time,
    1e3 * full_ATR["mass_size_distribution"].sum(dim="radius"),
    color=default_colors[3],
    label="All drops",
)

ax4.set_ylabel("LWC\n\nLWC in g/m^3")
# ax4.set_yscale("log")
ax4.legend()
fig.suptitle(f"Example Cloud and Cluster identification from {time_slice.start.date()}")
fig.savefig(fig_path / Path("example_cloud_cluster_identification.png"))